In [1]:
import pandas as pd
import numpy as np

## 1. 데이터 불러오기

In [7]:
data_cleansing = pd.read_excel('../데이터/크롤링 데이터/GlowPic_dataset_클렌징2022_10_27.xlsx', index_col=0)

data_sun_cream = pd.read_excel('../데이터/크롤링 데이터/GlowPic_dataset_선크림2022_11_04.xlsx', index_col=0)

data_cream = pd.read_excel('../데이터/크롤링 데이터/GlowPic_dataset_크림2022_10_30.xlsx', index_col=0)

data_skin = pd.read_excel('../데이터/크롤링 데이터/GlowPic_dataset_스킨,토너2022_11_04.xlsx', index_col=0)

data_lotion = pd.read_excel('../데이터/크롤링 데이터/GlowPic_dataset_로션,에멀젼2022_11_03.xlsx', index_col=0)


FileNotFoundError: [Errno 2] No such file or directory: '../데이터/크롤링 데이터/GlowPic_dataset_클렌징2022_10_27.xlsx'

In [3]:
# 행을 기준으로 다 병합 
total_data = pd.concat([data_cream, data_sun_cream, data_skin, data_lotion, data_cleansing])

In [4]:
# 엑셀로 저장
total_data.to_excel('상품 정보 데이터 총 합본(전처리전 병합만).xlsx', index=False)

In [5]:
# 저장한 파일 불러오기 
total_data = pd.read_excel('상품 정보 데이터 총 합본(전처리전 병합만).xlsx')

## 2. 데이터 전처리

### 2-1 기본 전처리 자동화

In [6]:
# 자동으로 전처리해주는 함수 구현 
def preprocessing(data):
    
    # 중복 삭제 
    data.drop_duplicates(subset='화장품명',keep='first', inplace=True)
    
    # 성분 개수 열 
    ingredient_count_feature = ['위험도 미정 성분 개수', '낮은 위험도 성분 개수', '중간 위험도 성분 개수', '높은 위험도 성분 개수']

    # 순위 관련 열 
    rank_feature = ['브랜드 내 순위', '카테고리 내 순위']
    
    # "개"라고 붙은 데이터 정수형으로 변환
    for feature in ingredient_count_feature:
        data[feature] = data[feature].apply(lambda x : int(x[:-1]))
        
    # 화장품 명 없는 행 삭제
    data.drop(data[data['화장품명'].isnull()].index, inplace=True)    
    
    # 화장품 명 앞에 (단종)인 데이터 삭제 
    data.drop(data[data['화장품명'].str.contains('(단종)')].index, inplace=True)    
    
    # 1mg가 1g 화장품 용량 단위 조정
    # 아직 해결 불가 
    
    # 가격이 "가격미정"인 행 삭제
    data.drop(data[data['가격']==' 가격미정'].index, inplace=True)
    
    # 가격 "원" 빼주고 정수형으로 변환(연속형)
    data['가격'] = data['가격'].apply(lambda x : int(x[:-1].replace(',', '').strip())) 
    
    # 만족도 '%" 제거후 정수형으로 변환(불연속형)
    data['만족도'] = data['만족도'].apply(lambda x : int(x[:-1]))
    
    
    # 순위 "위" 빼주고 정수형으로 변환(불연속형)-> Nan 값은 "순위 없음"으로 변경
    for feature in rank_feature:
        data[feature].fillna(0, inplace=True)
        data[feature] = data[feature].apply(lambda x : int(x[:-1]) if x != 0 else '순위 없음')
    
    # 카테고리 비어있는 값 "카테고리 미정"으로 변경
    data['카테고리'].fillna(0, inplace=True)
    data['카테고리'] = data['카테고리'].apply(lambda x : '카테고리 미정' if x == 0 else x)
    
    # 리뷰수 데이터 정수형으로 변경
    data['리뷰 수'] = data['리뷰 수'].apply(lambda x : int(x.replace(',', '').strip()))    
    
    # 성분 개수 열을 모두 더해서 "총 성분 개수" 열을 만든 다음에 값이 0인 행(성분이 하나도 없다는 뜻) 모두 삭제
    data['총 성분 개수'] = data['위험도 미정 성분 개수'] + data['낮은 위험도 성분 개수'] + data['중간 위험도 성분 개수'] + data['높은 위험도 성분 개수']      
    
    # 총 성분 개수가 0인 데이터 행 삭제
    data.drop(data[data['총 성분 개수']==0].index, inplace=True)
    
    # Null 값이 있는 행 삭제
    data.dropna(inplace= True)
    
    # 인덱스 초기화 
    data.reset_index(drop=True, inplace=True)    
    
    
    return data 

In [7]:
data = preprocessing(total_data)

/var/folders/7d/bm3zqtjx7xsb__yxj254h7kh0000gn/T/ipykernel_40858/2717977085.py:21: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data.drop(data[data['화장품명'].str.contains('(단종)')].index, inplace=True)


In [9]:
data.to_excel('상품 정보 데이터 총 합본(전처리 완료).xlsx', index=False)

In [12]:
data

,화장품명,브랜드명,브랜드 내 순위,용량,가격,평점,만족도,리뷰 수,카테고리,카테고리 내 순위,위험도 미정 성분 개수,낮은 위험도 성분 개수,중간 위험도 성분 개수,높은 위험도 성분 개수,키워드 카테고리,구성 성분 리스트,제품 설명,총 성분 개수
0,노르딕 버치 에너지 아이밤,시메스,1,10ml,19500,4.70,97,127,아이크림,1,1,33,3,1,크림,"['시어버터', '올리브오일', '정제수', '마이크로크리스탈린왁스', '비즈왁스'...",자연 유래 성분으로 케어하는 링클스팟\n\n- 핀란드산 자작나무잎추출물은 진정과 항...,38
1,더블 베리어 크림,큐어코드,2,80ml,38000,4.71,98,94,보습크림,1,4,25,0,0,크림,"['정제수', '글리세린', '펜틸렌글라이콜', '카프릴릭/카프릭트라이글리세라이드'...",뉴로마이드와 MES 보호막으로 피부 장벽을 탄탄하고 건강하게!\n\n- 피부 속 세...,29
2,모이스처 밸런싱 크림,백아율,2,50ml,32000,4.67,96,224,수분크림,1,3,43,4,2,크림,"['정제수', '글리세린', '부틸렌글라이콜', '하이드로제네이티드폴리데센', '펜...","닿는 순간 착, 쫀쫀한 찰떡 수분크림\n\n- 과학적으로 설계한 겔 속 수분과 영양...",52
3,리바이탈라이징 수프림 플러스 유스 파워 크림,에스티 로더,1,50ml,150000,4.71,99,84,안티에이징크림,1,2,42,12,1,크림,"['정제수', '다이메티콘', '글리세린', '아이소헥사데칸', '부틸렌글라이콜',...",깊은 보습감으로 피부 본연의 힘을 강화시켜 속부터 탄탄하게 차오르는 피부\n\n- ...,57
4,워터풀 히알루론산 크림,주미소,1,50g,24000,4.61,95,713,수분크림,2,3,33,0,0,크림,"['정제수', '부틸렌글라이콜', '글리세린', '카프릴릴메티콘', '펜틸렌글라이콜...",피부 갈증을 채우는 #수분길크림\n\n- 3중 히알루론산으로 피부 깊숙한 곳 부터 ...,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7302,클렌징바 [허브앤밀스],모나이트,8,120g,8000,5.00,100,2,클렌징바,135,0,19,5,1,클렌징,"['정제수', '스테아릭애씨드', '글리세린', '미리스틱애씨드', '라우릭애씨드'...","시어버터, 라벤더, 올리브오일로 만든 클렌징바",25
7303,퓨어마린 웰 클리어 클렌징 패드,아이멜리,8,70ea,21000,5.00,100,2,클렌징패드,40,1,36,0,1,클렌징,"['정제수', '다이프로필렌글라이콜', 'C12-13파레스-9', '폴리글리세릴-4...",귀찮은 클렌징을 한 번에! 원스텝 클렌징 패드\n\n- 한 장으로 빠르고 간편하게!...,38
7304,수블리마지 르 사봉,샤넬,206,115g,111000,5.00,100,2,클렌징바,128,1,17,6,1,클렌징,"['정제수', '프로필렌글라이콜', '소듐라우레스설페이트', '소듐스테아레이트', ...",투명한 광채를 밝혀 주는 프리미엄 리치 클렌징 솝\n\n- 꼼꼼하게 클렌징을 끝낸 ...,25
7305,누본셀 수노우 콜라겐 클렌저,블랑두부,3,150ml,25000,5.00,100,2,버블클렌저,87,0,30,1,2,클렌징,"['정제수', '글리세린', '다이소듐코코암포다이아세테이트', '부탄', '소듐클로...",쫀쫀하게 늘어나는 모찌타입 거품으로 피부 구석구석 올킬 세안\n\n- 피부와 유사한...,33


### 2-2 화장품별 성분 데이터셋 구축

- 화장품 명과 성분만 있는 데이터셋 구축

In [307]:
data['구성 성분 리스트'] = data['구성 성분 리스트'].apply(lambda x : np.pad(x.split('"')[0].split('[')[1].split(']')[0].replace("'", '').replace(' ', '').split(','),(0, data['총 성분 개수'].max()+ 1 -   
len(x.split('"')[0].split('[')[1].split(']')[0].replace("'", '').replace(' ', '').split(','))), 'constant', constant_values=0))

In [324]:
data_list = []
for i, name in enumerate(cosmetic_name):
    data_list.append(list(data['구성 성분 리스트'].values[i]))

In [330]:
ingredient_data = pd.DataFrame(data=data_list)
ingredient_data['화장품명'] = cosmetic_name
ingredient_data.set_index('화장품명', inplace=True)

In [333]:
ingredient_data = ingredient_data.transpose()

In [334]:
ingredient_data.to_excel('asda.xlsx', index=False)

- 가설: 용량이 낮을수록 화장품 가격이 높아진다.